# Homework:
    1) use MovieLens 1m data sets https://grouplens.org/datasets/movielens/1m/
    2) use any models from SURPRISE 
    3) get RMSE = 0.87 on test data set

In [1]:
import numpy as np
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic, SVD, BaselineOnly
from surprise.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
import pandas as pd

c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
#Read data set MovieLens

movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)

movies_with_ratings.dropna(inplace=True)

movies_with_ratings.drop('genres', axis=1, inplace=True)

label = LabelEncoder()

label.fit(movies_with_ratings.title.drop_duplicates())
label.fit(movies_with_ratings.title)

movies_with_ratings.title = label.transform(movies_with_ratings.title)

In [4]:
#movies_with_ratings = StandardScaler().fit_transform(movies_with_ratings)

In [4]:
movies_with_ratings.describe()

,movieId,title,userId,rating,timestamp
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,8.648988e+03,6.933018e+03,3.527086e+03,3.529272e+00,1.096036e+09
std,1.910014e+04,4.070101e+03,2.018424e+03,1.051919e+00,1.594899e+08
min,1.000000e+00,0.000000e+00,1.000000e+00,5.000000e-01,8.254999e+08
25%,9.030000e+02,3.431000e+03,1.813000e+03,3.000000e+00,9.658382e+08
50%,2.143000e+03,6.975000e+03,3.540000e+03,4.000000e+00,1.099263e+09
75%,4.641000e+03,1.058000e+04,5.233000e+03,4.000000e+00,1.217407e+09
max,1.306420e+05,1.402000e+04,7.120000e+03,5.000000e+00,1.427764e+09


In [5]:
#min rating == 0.5
movies_with_ratings['rating'].min()
#max rating == 0.5
movies_with_ratings['rating'].max()

5.0

In [6]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [7]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [8]:
# Train and Test data
trainset, testset = train_test_split(data, test_size=.15)

In [11]:
#def for user and item collaborative filtering


def knnBasicCrossvalidation():
        algo = KNNBasic(random_state=33, sim_options={'name': "pearson_baseline", 'user_based': False})
        cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

def svdCrossvalidation():
        algo = SVD()
        cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [12]:
%%time
knnBasicCrossvalidation()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8768  0.8789  0.8734  0.8802  0.8782  0.8775  0.0023  
MAE (testset)     0.6646  0.6673  0.6642  0.6679  0.6672  0.6662  0.0015  
Fit time          118.20  259.04  134.05  126.04  140.44  155.55  52.28   
Test time         134.03  224.55  213.43  211.20  257.86

In [13]:
%%time
svdCrossvalidation()

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8358  0.8339  0.8313  0.8326  0.8341  0.8335  0.0015  
MAE (testset)     0.6391  0.6381  0.6360  0.6376  0.6379  0.6377  0.0010  
Fit time          62.83   58.04   61.50   57.72   58.32   59.68   2.08    
Test time         4.85    2.69    2.65    2.93    2.96    3.22    0.83    
Wall time: 5min 27s
